# Neural Tangent Kernels

A few notes on the theory behind NTKs, based on  
- [Belkin et al, 2018](https://arxiv.org/abs/1812.11118)
-  [Jacot et al, 2018](http://arxiv.org/abs/1806.07572)
- [Lee et al, 2018](https://arxiv.org/abs/1711.00165)
-  [Lee et al, 2019](http://arxiv.org/abs/1902.06720)  
- [Chizat et al,2019](https://papers.nips.cc/paper/8559-on-lazy-training-in-differentiable-programming.pdf)



## Background 

- unclear why and how neural networks generalise so well
- more or less impossible to understand learning dynamics in nonlinear network  

### Bias/Variance Trade-Off in Neural networks (Belkin et al)
- bias/variance trade-off doesn't explain behaviour of neural networks:
- if increase size even beyound threshold required for convergence on training data, test error begins to drop again   

![u-shaped loss](./belkin.jpg)  

- authors introduce notion of double decent 
- motivation for study of networks in inifinte width limit 


### Infinite Width Neural Networks & Gaussian Processes  
In the infinite width limit, function specified by neural network can be understood as function drawn from a Gaussian Process.  
#### Background Reading:  
- Equivalence of infinite width random Nnets and GPs: [Neal, 1994a](https://www.cs.toronto.edu/~radford/ftp/pin.pdf)  
- Extension to Deep Nets: [Lee et al, 2018](https://arxiv.org/abs/1711.00165)    

Intuitive Explanation: Weights and biases are gaussian random variables. The sum of all inputs into a unit (i.e. its preactivations) is thus a sum of gaussian RVs which itself is a gaussian RV (CLT).     


#### Formally speaking
If $z_i = f_{i-1}(x_{i-1},\theta)$ denotes the preactivations at layer $i$, then 
$$
z_i \sim \mathcal{GP}(m(z(x)),K(z(x),z(x')))
$$
 as width goes to infinity.  
* Parameters have zero mean, i.e. 
$$m(z) = \mathbb{E}(z(x)) = 0$$
* Covariance matrix is given by kernel $$\kappa(x,x') = \langle z(x)z(x')\rangle$$
* Can be extended to deep nets via induction  

#### Interpretation
This is very cool from historical perspective: Nnets gained popularity in the 80s but computationally too demanding. Kernel methods took over. Now, nnets again on vogue. Good to know that there is a connection.

#### Problem
This only holds for a randomly initialised network where only the readout layer is trained. Therefore, doesn't really help in studying the learning dynamics under SGD.  


## Neural Tangent Kernels 
### *Neural Tangent Kernel (Jacot et al,2018)*  
#### Gual   
- we'd like to understand the learning dynamics of a neural network. Know great deal about dynamics in linear case (thanks Andrew!), but SGD in nonlinear network very hard to study 
- and Kernel Methods easier to interpret (?) but analogy only holds for random init network.


#### Proposed Solution
- NTK: kernel for nnets trained end-to-end with SGD! 

#### Gradient Flows: Derivation of NTK
Let's have a look at training dynamics under Gradient Descent. Standard Formula:    
$$
\theta_{t+1} = \theta_{t} - \eta \nabla_\theta L(\theta_t) \\
\frac{\theta_{t+1}-\theta{t}}{\eta} = -\nabla_\theta L(\theta_t)
$$
We assume an L2 loss:
$$
L(\theta) = \frac{1}{2}||y(\theta,x)-\hat{y}||
$$
If we assume an infinitesimally small learning rate $\eta$, we can interpret this as differential equation that tracks the change of the parameter vectors over time (**Gradient Flow**):
$$ 
\frac{d\theta(t)}{dt} = \dot{\theta} = -\nabla_\theta L(\theta_t) \\
 \dot{\theta} = -\nabla_\theta y(\theta)(y(\theta)-\hat{y})
$$
Now here comes the cool part: We're dealing with a neural network, so the chain rule of calculus applies! This means that we can derive the dynamics of the model outputs (i.e. dynamics in function space) induced by the dynamics of the weights very easily:
$$
\dot{y(\theta)} = \nabla_\theta y(\theta)^T\dot{\theta} = -\nabla_\theta y(\theta)^T\nabla_\theta y(\theta)(y(\theta)-\hat{y})
$$
Now, we assume that the gradient of the network function acts as a feature map $\phi(x)$, which implies that its dot product over all the data points is a valid kernel, the Neural Tangent Kernel (NTK):
$$ 
H(t) = \kappa_{NTK}(x,x') =  \langle -\nabla_\theta y(\theta(t),x)^T\nabla_\theta y(\theta(t),x) \rangle, \forall x,x' \in \mathbb{R}^d 
$$


#### Lazy Learning: NTK and the Infinite Width Limit
In the infinite width limit, we make an interesting observation: Weights change only very little during training, i.e. stay close to their random initialisation:
$$
\frac{||\theta(t)-\theta(0)||_2}{||\theta(0)||_2} \approx 0
$$ 
And hence, the kernel barely varies during training:
$$
\kappa_t(x,x') \approx \kappa_{t}(x,x'), \forall t 
$$ 
Let's come up with an intuitive explanation of this phenomenon: With infinitely many weights, very small changes lead to a big **net change** in the preactivations (i.e. linearity) of a hidden layer neuron. In contrast, with only very few weights, these have to change substantially to yield the same effect.  
Moreover, Jacot has shown that the kernel at initialisation, which depends on a random variable (weights drawn from iid gaussian) converges in probability to a deterministic value in the large weight limit, i.e. 
$$
\kappa_0(x,x') \approx \kappa_{NTK}(x,x')
$$
So now we know that the time-varying kernel is essentially identical to the NTK. Importantly, the network trained to convergence is equivalent to the kernel regression solution with an NTK
$$
f_{SGD}(x,\theta) \approx f_{NTK}(x,\theta) = \kappa_{NTK}(x,X)^T \kappa_{NTK}(X,X)^{-1}y
$$
where $X$ corresponds to the dataset s.t. $\kappa_{NTK}(x,X) = [\kappa_{NTK}(x,x_1),...,\kappa_{NTK}(x,x_n)]^T$


### Linear Approximation of Infinite Width Network (Lee et al,2019)  
At this point, it's still not clear to me how to study the dynamics of the network. However, we know that - in the NTK regime - the network parameters barely change during learning.  
insight: We can just apply a 1st-order Taylor expansion (hence a linear approximation) to the network function w.r.t. its parameters around its initialisation. Now, the non-linear network is a linear function of the weights, yet still non-linear in its inputs:
$$
f(x,\theta) \approx f(x,\theta_0) + \nabla_\theta f(x,\theta_0)^T(\theta-\theta_0)
$$
And here comes the connection to NTKs: We can understand this as a linear function of x with a non-linear basis function (feature map) of the inputs, which is simply the gradient on the weights:
$$
\phi(x) = \nabla_\theta f(x,\theta_0)
$$
so the function becomes: 
$$
f(x,\theta) \approx f(x,\theta_0) + \phi(x)^T(\theta-\theta_0)
$$
This feature map can be understood as a kernel on the inputs, for which we know its form:
$$
\kappa(x,x') = \phi(x)^T\phi(x') = \nabla_\theta f(x,\theta_0)^T \nabla_\theta f(x,\theta_0)
$$
(Is this correct?)
Now, the training dynamics are tractable, as we have a linear function of the weights.
Todo: brush up my understanding of dual view and the kernel trick (https://alliance.seas.upenn.edu/~cis520/dynamic/2017/wiki/index.php?n=Lectures.Kernels#toc5) 

## Lazy Learning and NTK Regime on standard Neural Networks (chizat et al, 2019)

All above is pretty cool, but only holds in the infinite width limit. I understand intuitively why. With many weights, each contributes a little, thus ensemble has large impact on output, thus no need for dramatic changes (highly distributed code).
But in reality, width is obs. finite. How to apply these findings to real nnets?
**Chizat et al argue that a finite-width network can be pushed into the NTK regime by chosing an appropriate explicit scaling factor (ofwhat? outputs, weights?).**They demonstrate that any parametric model (not only nnets) can be trained in lazy regime if its outputs are close to zero after random initialisation. If I understand correctly, this can be achieved with LeCun initialisation (variance scales with network width?)

## Our Work
*connection to our work*  
- NTK all the rage as it draws connections between nnets and kernel methods and provides opportunity to study learning dynamics even in nonlinear networks.
- with the right initialisation, even finite width networks can be pushed into NTK regime and can then be studied using these tools.
- But I've read somewhere that only a few networks actually operate in this regime, and many successful practical applications don't.
- However, appealing from theoretical point of view, as they should in theory generalise very well
- our contribution: in continual learning, sth else happens, nnet actually learns compression with units tuned to individual tasks (localist code). we know that this is beneficial to prevent interference. 



